In [9]:
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import random as r

In [10]:
def generator():
    lst = list(np.random.choice((1, 2, 3), size=100, p=(0.1, 0.65, 0.25)))
    lst_std = np.random.standard_normal(100) + 5
    a = 0
    for i, item in enumerate(lst):
        if item == 1:
            a = np.random.choice((1, 2, 3, 4, 5), p=(0.78, 0.05, 0.03, 0.12, 0.02))
        elif item == 2:
            a = np.random.choice((1, 2, 3, 4, 5), p=(0.17, 0.6, 0.11, 0, 0.12))
        elif item == 3:
            a = np.random.choice((1, 2, 3, 4, 5), p=(0.77, 0, 0.06, 0.09, 0.08))
        len_message = random.randint(19, 62)
        lst[i] = [lst[i], a, len_message, lst_std[i]]
    return lst

In [11]:
class Data:
    def __init__(self, n):
        self.arr = n
        self.mean = np.mean(self.arr)
        self.dis = np.var(self.arr)
        self.otkl = np.std(self.arr)

    def __repr__(self):
        return f"Мат ожидание:{self.mean}, Дисперсия:{self.dis}, Ср кв отклонение:{self.otkl}, Интенсивность:{1 / self.mean}"

In [12]:
class Perem:
    def __init__(self):
        self.type = 0
        self.adr = 0
        self.len = 0
        self.time = 0

    def __repr__(self):
        return f"{self.type} {self.adr} {self.len} {self.time:.7}"

In [13]:
def create_data():
    x = np.random.standard_normal(100) + 5
    arr = []
    for i in range(100):
        temp = Perem()
        num = r.random()
        if 0 <= num <= 0.1:
            temp.type = 1
            temp.adr = np.random.choice(range(1, 6), p=(0.78, 0.05, 0.03, 0.12, 0.02))
        elif 0.1 < num <= 0.75:
            temp.type = 2
            temp.adr = np.random.choice(range(1, 6), p=(0.17, 0.6, 0.11, 0, 0.12))
        elif 0.75 < num <= 1:
            temp.type = 3
            temp.adr = np.random.choice(range(1, 6), p=(0.77, 0, 0.06, 0.09, 0.08))
        # elif 0.65 < num <= 1:
        #     temp.type = 4
        #     temp.adr = np.random.choice(range(1, 6), p=(0.02, 0.72, 0.01, 0.07, 0.18))
        temp.len = random.randint(19, 62)
        temp.time = x[i]
        arr.append(temp)
    return arr

In [14]:
class Req:
    counter = 1

    def __init__(self, t, m):
        self.id = Req.counter
        self.create = t
        self.start = 0
        self.end = 0
        self.out = 0
        self.spent = math.ceil(m.len / 6)
        self.data = m
        Req.counter += 1

    def calc_out(self):
        self.out += self.start - self.create

    def __repr__(self):
        return f"{self.id} {self.create} {self.start} {self.end} {self.out} {self.spent}"

In [15]:
class Que:
    def __init__(self):
        self.countq = [0, 0, 0]
        self.request = []
        self.speed = 6

    def process(self, arr1):
        arr = arr1.copy()
        time = int(arr[0].time * 60)
        while len(arr) != 0:
            mint = min(arr, key = lambda x: x.type).type
            i = 0
            while i != len(arr):
                if arr[i].type > mint:
                    self.countq[arr[i].type - 1] += 1
                    if not any(x for x in self.request if x.data == arr[i]):
                        self.request.append(Req(time, arr[i]))
                        time += 1
                else:
                    if any(x for x in self.request if x.data == arr[i]):
                        for x in self.request:
                            if x.data == arr[i]:
                                time += 1
                                x.start = time
                                x.end = x.start + x.spent
                                time = x.end
                                del arr[i]
                                i -= 1
                                break
                    else:
                        temp = Req(time, arr[i])
                        time += 1
                        temp.start = time
                        temp.end = temp.start + temp.spent
                        time = temp.end
                        self.request.append(temp)
                        del arr[i]
                        i -= 1
                i += 1

    def razn(self, arr):
        new = []
        for i in range(len(arr) - 1):
            new.append(arr[i + 1] - arr[i])
        return new

    def calc(self, arr):
        param = [
            {'timeouts': [], 'w': 0, 'aga': [], 'y': 0, 'times': [], 'averageTime/expectedValue': 0, "l": 0,
             "dispersion": 0, "standardDeviation": 0, "intensity": 0, "averageCnt": 0, "pk": 0, "n": 0, "p": 0, "u": 0}
            for x in range(3)
        ]
        print("1)")
        coun = [0 for i in range(4)]
        for i in arr:
            coun[i.type - 1] += 1
        a = 0
        arrange = []
        for j in range(1, 4):
            arrange.append(np.mean([i.len for i in arr if i.type == j]))
        for i in arrange:
            print(f"{i:.5}", end = " ")
        print()
        for i in range(3):
            param[i]["averageCnt"] = (coun[i] + self.countq[i]) / 100
            a += param[i]["averageCnt"]
            print(param[i]["averageCnt"], end = " ")
        print("\n2)")
        for i in self.request:
            param[i.data.type - 1]["aga"].append(i.create)
            param[i.data.type - 1]["times"].append(i.spent)
            param[i.data.type - 1]["timeouts"].append(i.out)
        w = 0
        u = 0
        l = 0
        r = 0
        for i in param:
            i["aga"] = self.razn(i["aga"])
            i["averageTime/expectedValue"] = np.mean(i["times"])
            i["dispersion"] = np.var(i["times"])
            i["standardDeviation"] = np.std(i["times"])
            i["intensity"] = 1 / i["averageTime/expectedValue"]
            i["pk"] = i["averageCnt"] / i["intensity"]
            i["n"] = abs(1 - i["pk"])
            i["t"] = np.mean(i["aga"])
            i["y"] = 1 / i["t"]
            i["p"] = i["y"] / a
            i["w"] = np.mean(i["timeouts"])
            i["l"] = i["w"] * i["y"]
            i["u"] = i["averageTime/expectedValue"]
            w += i["p"] * i["w"]
            u += i["p"] * i["u"]
            l += i["y"] * i["w"]
            r += i["pk"]
        for i in param:
            print(f"{i['l']:.3f}", end = " ")
        print(f"\n{w:.3f}\n{u:.3f}\n{l:.3f}")
        print("3)")
        for i in ["averageCnt", "pk", "n", "r", "u", "intensity"]:
            if i == "r":
                print(f"{r:.3f}")
            else:
                for j in range(3):
                    print(f"{param[j][i]:.3f}", end = " ")
                print()

In [16]:
queue = Que()
arr = sorted(create_data(), key = lambda x: x.time)
queue.process(arr)

for i in queue.request:
    i.calc_out()
    print(i)
queue.calc(arr)
print()

1 147 348 355 201 7
2 148 149 158 1 9
3 158 356 363 198 7
4 159 876 885 717 9
5 160 161 166 1 5
6 166 364 371 198 7
7 167 886 890 719 4
8 168 372 381 204 9
9 169 170 175 1 5
10 175 382 388 207 6
11 176 389 398 213 9
12 177 891 896 714 5
13 178 399 404 221 5
14 179 897 906 718 9
15 180 907 911 727 4
16 181 405 414 224 9
17 182 415 423 233 8
18 183 424 433 241 9
19 184 434 441 250 7
20 185 442 447 257 5
21 186 448 456 262 8
22 187 457 465 270 8
23 188 466 473 278 7
24 189 474 478 285 4
25 190 912 917 722 5
26 191 479 488 288 9
27 192 489 499 297 10
28 193 918 923 725 5
29 194 500 507 306 7
30 195 508 513 313 5
31 196 514 523 318 9
32 197 198 208 1 10
33 208 524 530 316 6
34 209 531 541 322 10
35 210 211 217 1 6
36 217 542 551 325 9
37 218 219 223 1 4
38 223 552 562 329 10
39 224 225 230 1 5
40 230 563 567 333 4
41 231 568 574 337 6
42 232 233 243 1 10
43 243 575 585 332 10
44 244 924 929 680 5
45 245 586 592 341 6
46 246 593 604 347 11
47 247 605 616 358 11
48 248 930 939 682 9
49 249 61